In [14]:
## Importing standards libraries

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import time

## Importing libraries for the Gaussian Mixture Model

from scipy.stats import multivariate_normal
from scipy.stats import norm
import seaborn as sns

## Autograd packages

## Convergence metrics

from scipy.stats import wasserstein_distance
from scipy.stats import entropy # KL-divergence
import ot #Optimal Transport


## Import functions for the experiment : 

from experiment_functions import *
from IPLA_Exp_Functions import *


ModuleNotFoundError: No module named 'experiment_functions'

In this experiment, we consider a Parametric latent variable models (LVM) in a specific setting. In this context, we will be able to do a fair comparison between several algorithms that aim to estimate the Maximum Marginal Likelihood of the model. 
\\ \\
Let us consider a generic latent variable model as $p_\theta(x,y)$, parameterised by $\theta \in \mathbb{R}^{d_\theta}$, for fixed data $y \in \mathbb{R}^{d_y}$, and latent variables $x \in \mathbb{R}^{d_x}$. This model is considered for fixed observed data $y$, thus formally we see the statistical model as a real-valued mapping $p_\theta(x,y) : \mathbb{R}^{d_x} \times \mathbb{R}^{d_\theta} \rightarrow \mathbb{R}$. The task we are interested in is to estimate the parameter $\theta$ that explains the fixed dataset $y$. Often, this task is achieved via the maximum likelihood estimation (MLE). In accordance with this, in our setting, due to the presence of latent variables, we aim at finding the maximum marginal likelihood estimate (MMLE), more precisely, our problem takes the form

\begin{equation}
\theta^\ast \in \arg \max_{\theta \in \mathbb{R}^{d_\theta}} \log p_\theta(y),
\end{equation}

where $p_\theta(y) := \int p_\theta(x,y) \, dx$ is the marginal likelihood.
\\ 
The main difficulty is that, very often, this integral is intractable. This is where the different algorithms studied appear. 
\\ \\
With the notation $U(\theta, x) = -\log p_\theta(x,y)$, the $\theta$-gradient of $\log p_\theta(y)$ is given as

\begin{equation}
\nabla_\theta \log p_\theta(y) = - \frac{\int \nabla_\theta U(\theta, x) p_\theta(x, y) \, dx}{p_\theta(y)} = - \int \nabla_\theta U(\theta, x) p_\theta(x \mid y) \, dx,
\end{equation}


\end{justify}
